In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import warnings
import nltk
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('gujarati-train.csv', encoding="utf-8")

In [3]:
def count_length():
    df['word_count'] = df['headline'].apply(lambda x: len(str(x).split(" ")))

In [4]:
count_length()

In [5]:
df.head(10)

,headline,Label,word_count
0,IPL Auction 2022: 12-13 ફેબ્રુઆરીએ કેટલા વાગ્ય...,Sports,10
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, 155kphન...",Sports,10
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11
4,ક્યાં રમાશે IPL 2022ના મેચ? સૌરવ ગાંગુલીએ આપ્ય...,Sports,10
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11
6,"IND vs WI: ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી...",Sports,15
7,IPL 2022: હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુ...,Sports,12
8,IPL 2022ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ ...,Sports,17
9,IPL 2022: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે...,Sports,12


In [6]:
f = open("Gujarati_stopwords.txt", "r",encoding="utf8")
stopwords_gu = []
for i in f:
    i=i.split("\n")
    stopwords_gu.append(str(i[0]))
stopwords_en = stopwords.words('english')
punctuations = [".",",","?","/","<",">","!",":",";","'","[","]","{","}","(",")","-","@","#","$","%","^","&","*"]
stopwords_lst = stopwords_gu + stopwords_en + punctuations

In [7]:
df['label_id'] = df.Label.factorize()[0]

def processText(text):
    text = text.lower()
    text = re.sub('[a-zA-Z0-9]', '',text)
    text = re.sub("((www.[^s]+)|(https?://[^s]+))",'',text)
    text = re.sub("@[^s]+",'',text)
    text = re.sub("[s]+", ' ', text)
    text = re.sub(r"#([^s]+)", r'', text)
    text = text.strip('"')
    return text 

In [8]:
for i in range(len(df)):
    df['headline'][i] = processText(df['headline'][i])
count_length()

In [9]:
df.head(10)

,headline,Label,word_count,label_id
0,: - ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓન...,Sports,10,0
1,રોહિત-કોહલીના વિવાદ પર ગાવસ્કરે કર્યો મોટો ખુલ...,Sports,10,0
2,"પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ, ની સ્પિ...",Sports,10,0
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,11,0
4,ક્યાં રમાશે ના મેચ? સૌરવ ગાંગુલીએ આપ્યો આ જવાબ,Sports,10,0
5,"ટીમ ઈન્ડિયાનો આ ખેલાડી વેડફી રહ્યો છે ટેલેન્ટ,...",Sports,11,0
6,": ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ, ફરી પ્રથમ ...",Sports,15,0
7,": હરાજીમાં આ ખેલાડીઓનું વેચાવવું છે મુશ્કેલ, ...",Sports,12,0
8,ના મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે છે આ વિદેશી...,Sports,17,0
9,: ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં છે મેગા ઓ...,Sports,12,0


In [10]:
from indicnlp.tokenize import indic_tokenize
def tokenization(indic_string):
    tokens = []
    for t in indic_tokenize.trivial_tokenize(indic_string):
        tokens.append(t)
    return tokens

In [11]:
df['headline'] = df.headline.apply(lambda x: tokenization(x))
df.head(10)

,headline,Label,word_count,label_id
0,"[:, -, ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલ...",Sports,10,0
1,"[રોહિત, -, કોહલીના, વિવાદ, પર, ગાવસ્કરે, કર્યો...",Sports,10,0
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, ,,...",Sports,10,0
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,11,0
4,"[ક્યાં, રમાશે, ના, મેચ, ?, સૌરવ, ગાંગુલીએ, આપ્...",Sports,10,0
5,"[ટીમ, ઈન્ડિયાનો, આ, ખેલાડી, વેડફી, રહ્યો, છે, ...",Sports,11,0
6,"[:, ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, ,, ફરી,...",Sports,15,0
7,"[:, હરાજીમાં, આ, ખેલાડીઓનું, વેચાવવું, છે, મુશ...",Sports,12,0
8,"[ના, મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, છે, આ,...",Sports,17,0
9,"[:, ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, છે...",Sports,12,0


In [12]:
for i in range(len(df)):
    df['headline'][i]=[ele for ele in df['headline'][i] if ele not in (stopwords_lst)]
count_length()
df.head(10)

,headline,Label,word_count,label_id
0,"[ફેબ્રુઆરીએ, કેટલા, વાગ્યાથી, લાગશે, ખેલાડીઓની...",Sports,6,0
1,"[રોહિત, કોહલીના, વિવાદ, ગાવસ્કરે, કર્યો, મોટો,...",Sports,10,0
2,"[પાકિસ્તાની, ફાસ્ટ, બોલરને, કરાયો, સસ્પેંડ, સ્...",Sports,7,0
3,"[એશિઝમાં, કારમી, હારને, લીધે, ગિલ્સ, બાદ, મુખ્...",Sports,10,0
4,"[ક્યાં, રમાશે, મેચ, સૌરવ, ગાંગુલીએ, જવાબ]",Sports,6,0
5,"[ટીમ, ઈન્ડિયાનો, ખેલાડી, વેડફી, રહ્યો, ટેલેન્ટ...",Sports,9,0
6,"[ભારતીય, ટીમમાં, કોરોનાનો, વિસ્ફોટ, મેચ, નહી, ...",Sports,10,0
7,"[હરાજીમાં, ખેલાડીઓનું, વેચાવવું, મુશ્કેલ, કરોડ...",Sports,7,0
8,"[મેગા, ઓક્શનમાં, અનસોલ્ડ, રહી, શકે, વિદેશી, ખે...",Sports,10,0
9,"[ક્યા, દેશના, કેટલા, ખેલાડી, લઇ, રહ્યાં, મેગા,...",Sports,8,0


In [13]:
df['headline'] = df['headline'].str.join(" ")
df.head(10)

,headline,Label,word_count,label_id
0,ફેબ્રુઆરીએ કેટલા વાગ્યાથી લાગશે ખેલાડીઓની બોલી,Sports,6,0
1,રોહિત કોહલીના વિવાદ ગાવસ્કરે કર્યો મોટો ખુલાસો...,Sports,10,0
2,પાકિસ્તાની ફાસ્ટ બોલરને કરાયો સસ્પેંડ સ્પિડથી ...,Sports,7,0
3,એશિઝમાં કારમી હારને લીધે ગિલ્સ બાદ મુખ્ય કોચને...,Sports,10,0
4,ક્યાં રમાશે મેચ સૌરવ ગાંગુલીએ જવાબ,Sports,6,0
5,ટીમ ઈન્ડિયાનો ખેલાડી વેડફી રહ્યો ટેલેન્ટ ગાવસ્...,Sports,9,0
6,ભારતીય ટીમમાં કોરોનાનો વિસ્ફોટ મેચ નહી રમી શકે...,Sports,10,0
7,હરાજીમાં ખેલાડીઓનું વેચાવવું મુશ્કેલ કરોડોમાં ...,Sports,7,0
8,મેગા ઓક્શનમાં અનસોલ્ડ રહી શકે વિદેશી ખેલાડી કર...,Sports,10,0
9,ક્યા દેશના કેટલા ખેલાડી લઇ રહ્યાં મેગા ઓક્શનમાં,Sports,8,0


In [14]:
# set parameters:
max_features = 20000
maxlen = 400   #400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 15 #25

In [15]:
X = df['headline'].values
y = df['label_id']
y = to_categorical(y, num_classes=5)
y.shape

(37779, 5)

In [16]:
print('Loading data...')
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

x_train = tokenizer.texts_to_sequences(x_train)
x_test  = tokenizer.texts_to_sequences(x_test)

x_train = pad_sequences(x_train, padding='post', maxlen=maxlen)
x_test  =  pad_sequences(x_test, padding='post', maxlen=maxlen)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
print('Pad sequences (samples x time)')

x_train = pad_sequences(x_train, maxlen=maxlen)
x_test  =  pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

Loading data...
30223 train sequences
7556 test sequences
Pad sequences (samples x time)
x_train shape: (30223, 400)
x_test shape: (7556, 400)


In [17]:
test_classes = []
for i in range(len(y_test)):
    for j in range(len(y_test[i])):
        if y_test[i][j] == 1:
            if j==0:
                test_classes.append(0)
            if j==1:
                test_classes.append(1)
            if j==2:
                test_classes.append(2)
            if j==3:
                test_classes.append(3)
            if j==4:
                test_classes.append(4)

In [18]:
import json
tokenizer_json = tokenizer.to_json()
with open('gu_tokenizerfinal.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(tokenizer_json, ensure_ascii=True))

In [19]:
print('Build model...')
model = Sequential()

# we start off with an efficient embedding layer which maps
# our vocab indices into embedding_dims dimensions
model.add(Embedding(max_features,
                    embedding_dims,
                    input_length=maxlen))
model.add(Dropout(0.2))

# we add a Convolution1D, which will learn filters
# word group filters of size filter_length:
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu',
                 strides=1))
# we use max pooling:
model.add(GlobalMaxPooling1D())

# We add a vanilla hidden layer:
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))

# We project onto a single unit output layer, and squash it with a sigmoid:
model.add(Dense(5))
model.add(Activation('sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 400, 50)           1000000   
                                                                 
 dropout (Dropout)           (None, 400, 50)           0         
                                                                 
 conv1d (Conv1D)             (None, 398, 250)          37750     
                                                                 
 global_max_pooling1d (Globa  (None, 250)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 250)               62750     
                                                                 
 dropout_1 (Dropout)         (None, 250)               0         
                                         

In [20]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          validation_data=(x_test, y_test))

Epoch 1/15
945/945 [==============================] - 110s 116ms/step - loss: 0.5582 - accuracy: 0.7893 - val_loss: 0.2792 - val_accuracy: 0.9027
Epoch 2/15
945/945 [==============================] - 99s 104ms/step - loss: 0.1483 - accuracy: 0.9494 - val_loss: 0.2906 - val_accuracy: 0.9048
Epoch 3/15
945/945 [==============================] - 101s 107ms/step - loss: 0.0648 - accuracy: 0.9784 - val_loss: 0.3317 - val_accuracy: 0.9029
Epoch 4/15
945/945 [==============================] - 106s 112ms/step - loss: 0.0367 - accuracy: 0.9884 - val_loss: 0.4262 - val_accuracy: 0.8913
Epoch 5/15
945/945 [==============================] - 94s 99ms/step - loss: 0.0258 - accuracy: 0.9915 - val_loss: 0.4901 - val_accuracy: 0.8945
Epoch 6/15
945/945 [==============================] - 101s 107ms/step - loss: 0.0197 - accuracy: 0.9941 - val_loss: 0.5185 - val_accuracy: 0.8945
Epoch 7/15
945/945 [==============================] - 97s 103ms/step - loss: 0.0169 - accuracy: 0.9956 - val_loss: 0.5472 - val

In [21]:
loss, Accuracy = model.evaluate(x_test, y_test)

print("Test Loss:", loss)
print("Test Accuracy:", Accuracy)

237/237 [==============================] - 6s 25ms/step - loss: 0.8674 - accuracy: 0.8851
Test Loss: 0.8673853278160095
Test Accuracy: 0.8851243853569031


In [22]:
model.save('GU_CNN_final.h5')
model.save_weights('GU_CNN_weights_final.h5')

In [23]:
predicted_categories = []
preds = model.predict(x_test)
for i in preds:
    pred_classes = np.argsort(i)[-1:][::-1]
    predicted_categories.append(pred_classes[0])

In [24]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
report = classification_report(test_classes, predicted_categories)
print(report)

              precision    recall  f1-score   support

           0       0.86      0.94      0.90      1580
           1       0.85      0.82      0.84      1465
           2       0.83      0.87      0.85      1491
           3       0.97      0.89      0.93      1543
           4       0.93      0.90      0.91      1477

    accuracy                           0.89      7556
   macro avg       0.89      0.88      0.88      7556
weighted avg       0.89      0.89      0.89      7556



In [25]:
mat = confusion_matrix(test_classes, predicted_categories)
mat

array([[1482,   17,   25,   17,   39],
       [  61, 1207,  167,    6,   24],
       [  34,  134, 1292,    6,   25],
       [  83,   21,   46, 1374,   19],
       [  60,   40,   29,   15, 1333]], dtype=int64)

In [26]:
import json
from keras_preprocessing.text import tokenizer_from_json

def load():
    model = load_model('GU_CNN_final.h5')
    maxlen = 400
    id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

    with open('gu_tokenizerfinal.json') as f:
        data = json.load(f)
        tokenizer = tokenizer_from_json(data)
load()

In [27]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input1 =["શેરમાર્કેટમાં કડાકો:10 દિવસમાં Mcap 15 લાખ કરોડ ઘટ્યું; સેન્સેક્સ 678 પોઇન્ટ તૂટ્યો, ઇન્વેસ્ટર્સે 3 દિવસમાં 6.15 લાખ ગુમાવ્યાં"]

input_sequences = tokenizer.texts_to_sequences(my_input1)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result1 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result1[c] = round(p*100,2)
result1

{'business': 100.0,
 'tech': 2.06,
 'entertainment': 0.0,
 'sports': 0.0,
 'politics': 0.0}

In [28]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input2 =["સાઉથ આફ્રિકન સ્પિનર શેન વ્હાઈટહેડે ઈનિંગમાં ૧૦ વિકેટ ઝડપી ઈતિહાસ રચ્યો"]

input_sequences = tokenizer.texts_to_sequences(my_input2)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result2 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result2[c] = round(p*100,2)
result2

{'sports': 100.0,
 'politics': 0.0,
 'tech': 0.0,
 'entertainment': 0.0,
 'business': 0.0}

In [29]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input3 =["SRKના દીકરાની ઘરવાપસી LIVE:આર્યન ખાનને 'મન્નત'થી આર્થર રોડ જેલનું 13 કિલોમીટરનું અંતર કાપતાં 28 દિવસ થયા"]

input_sequences = tokenizer.texts_to_sequences(my_input3)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result3 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result3[c] = round(p*100,2)
result3

{'entertainment': 100.0,
 'tech': 82.7,
 'business': 65.59,
 'sports': 0.0,
 'politics': 0.0}

In [30]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input4 =["લોન્ચ થયો Oppo નો Waterproof સ્માર્ટફોન, ધાંસૂ કેમેરા સાથે મળશે ગજબના ફીચર્સ"]

input_sequences = tokenizer.texts_to_sequences(my_input4)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result4 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result4[c] = round(p*100,2)
result4

{'tech': 100.0,
 'sports': 0.01,
 'business': 0.0,
 'entertainment': 0.0,
 'politics': 0.0}

In [31]:
# Here's how to generate a prediction on individual examples
#text_labels = encoder.classes_ 
id_to_category = {0:'sports', 1:'tech', 2:'business', 3:'politics',
                  4:'entertainment'}

my_input5 =["ઉત્તરાખંડમાં ભાજપને મોટો ઝટકો, કેબિનેટ મંત્રી પદેથી રાજીનામું આપીને આ નેતા કોંગ્રેસમાં થયા સામેલસ"]

input_sequences = tokenizer.texts_to_sequences(my_input5)
input_pad = pad_sequences(input_sequences, padding='post', maxlen=maxlen)

preds = model.predict(input_pad)[0]

pred_classes = np.argsort(preds)[-5:][::-1]

classes = [id_to_category[i] for i in pred_classes]
props   = preds[pred_classes]

result5 = {}
for c, p in zip(classes, props):
    #result.append("{} {:.2f} %".format(c,p*100))
    result5[c] = round(p*100,2)
result5

{'politics': 100.0,
 'business': 0.92,
 'sports': 0.32,
 'entertainment': 0.2,
 'tech': 0.0}